In [6]:
%config Completer.use_jedi = False

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
from utils import *
from sklearn.metrics import accuracy_score, confusion_matrix

In [8]:
# import the necessary packages
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

In [9]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Data preparation

In [10]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [11]:
train_images.shape

(50000, 32, 32, 3)

In [12]:
test_images.shape

(10000, 32, 32, 3)

In [13]:
train_images.astype('float32') 
test_images.astype('float32') 

train_images = train_images/255.
test_images = test_images/255.

In [14]:
train_labels_cat = to_categorical(train_labels)
test_labels_cat = to_categorical(test_labels)

# Vanilla network

In [13]:
def build_vanilla_model(shape, class_num):
    # specify the inputs for the feature extractor network
    inputs = Input(shape)

    x = Conv2D(32, (3, 3), activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(64, (2, 2), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.2)(x)
    
    x = Conv2D(128, (2, 2), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.2)(x)
    
    x = Flatten()(x)

    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    outputs = Dense(class_num, activation='softmax')(x)

    # build the model
    model = Model(inputs, outputs)
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model

### model on the whole dataset

learning

In [11]:
# model = build_vanilla_model(train_images[0].shape, 10)

In [12]:
# history = model.fit(train_images, train_labels_cat, epochs=200, batch_size=512, 
#                     validation_data=(test_images, test_labels_cat))

In [13]:
# print_history_chart(history)

In [14]:
# model.save('vanilla_cifar10.h5')

evaluation

In [15]:
model = tf.keras.models.load_model("vanilla_cifar10.h5")

In [16]:
results = full_evaluate_differencing(model, test_images, test_labels)

Accuracy: 0.9281099999999999
Confusion matrix:
 [[8645  363]
 [ 336  656]]
F1-score: 0.6524117354549974


### models on the smaller datasets

#### 50% of the training data

learning

In [17]:
# model = build_vanilla_model(train_images[0].shape, 10)

In [18]:
# train_images_50p = train_images[:len(train_images)//2]
# train_labels_cat_50p = train_labels_cat[:len(train_labels_cat)//2]

In [19]:
# history = model.fit(train_images_50p, train_labels_cat_50p, epochs=300, batch_size=512, 
#                     validation_data=(test_images, test_labels_cat))

In [20]:
# print_history_chart(history)

In [21]:
# model.save('vanilla_50p_cifar10.h5')

evaluation

In [22]:
model = tf.keras.models.load_model("vanilla_50p_cifar10.h5")

In [23]:
results_vanilla_50p = full_evaluate_differencing(model, test_images, test_labels)

Accuracy: 0.91859
Confusion matrix:
 [[8575  433]
 [ 396  596]]
F1-score: 0.5898070262246413


#### 10% of the training data

learning

In [24]:
# model = build_vanilla_model(train_images[0].shape, 10)

In [25]:
# train_images_10p = train_images[:len(train_images)//10]
# train_labels_cat_10p = train_labels_cat[:len(train_labels_cat)//10]

In [26]:
# history = model.fit(train_images_10p, train_labels_cat_10p, epochs=400, batch_size=512, 
#                     validation_data=(test_images, test_labels_cat))

In [27]:
# print_history_chart(history)

In [28]:
# model.save('vanilla_10p_cifar10.h5')

evaluation

In [29]:
model = tf.keras.models.load_model("vanilla_10p_cifar10.h5")

In [30]:
results_vanilla_10p = full_evaluate_differencing(model, test_images, test_labels)

Accuracy: 0.8890500000000001
Confusion matrix:
 [[8447  561]
 [ 544  448]]
F1-score: 0.44777611194402794


# Siemese network

In [16]:
# def build_siamese_vanilla_model(shape):
#     inputs1 = Input(shape)
#     inputs2 = Input(shape)
    
#     def one_side(inputs):
#         x = Conv2D(32, (3, 3), activation="relu")(inputs)
#         x = MaxPooling2D(pool_size=(2, 2))(x)

#         x = Conv2D(64, (2, 2), activation="relu")(x)
#         x = MaxPooling2D(pool_size=(2,2))(x)
#         x = Dropout(0.2)(x)

#         x = Conv2D(128, (2, 2), activation="relu")(x)
#         x = MaxPooling2D(pool_size=(2,2))(x)
#         x = Dropout(0.2)(x)

#         x = Flatten()(x)
        
#         return x
    
#     x1 = one_side(inputs1)
#     x2 = one_side(inputs2)
    
#     x = concatenate([x1, x2])
    
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.1)(x)
    
#     x = Dense(16, activation='relu')(x)
    
#     outputs = Dense(1, activation='sigmoid')(x)

#     # build the model
#     model = Model([inputs1, inputs2], outputs)

#     return model

# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy', tf.keras.metrics.AUC()])


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.datasets import mnist
import numpy as np
import tensorflow.keras.backend as K


def build_siamese_model(inputShape):
    inputs = Input(inputShape)

    x = Conv2D(32, (3, 3), activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)

    x = Conv2D(64, (3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)
    
    x = Conv2D(128, (3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)

#     pooledOutput = GlobalAveragePooling2D()(x)
    pooledOutput = Flatten()(x)
    outputs = Dense(32)(pooledOutput)
    

    model = Model(inputs, outputs)

    return model


def euclidean_distance(vectors):
	# unpack the vectors into separate lists
	(featsA, featsB) = vectors

	# compute the sum of squared distances between the vectors
	sumSquared = K.sum(K.square(featsA - featsB), axis=1,
		keepdims=True)

	# return the euclidean distance between the vectors
	return K.sqrt(K.maximum(sumSquared, K.epsilon()))


# imgA = Input(shape=train_images[0].shape)
# imgB = Input(shape=train_images[0].shape)
# featureExtractor = build_siamese_model(train_images[0].shape)
# featsA = featureExtractor(imgA)
# featsB = featureExtractor(imgB)

# # finally, construct the siamese network
# distance = Lambda(euclidean_distance)([featsA, featsB])

# x = concatenate([distance, featsA, featsB])
# outputs = Dense(64, activation="relu")(x)
# outputs = Dense(1, activation="sigmoid")(outputs)
# model = Model(inputs=[imgA, imgB], outputs=outputs)

# adam = tf.keras.optimizers.Adam(
#     learning_rate=0.02,
# )


# model.compile(loss="binary_crossentropy", optimizer="adam",
#               metrics=["accuracy", tf.keras.metrics.AUC()])

### model on the whole datasets

learning

In [17]:
# model = build_siamese_vanilla_model(train_images[0].shape)

In [18]:
# adam = tf.keras.optimizers.Adam(
#     learning_rate=0.05,
# )

# model.compile(optimizer=adam,
#               loss='binary_crossentropy',
#               metrics=['accuracy', tf.keras.metrics.AUC()])

In [19]:
# a, b = 1, 2

# X1 = np.concatenate([train_images,train_images])
# X2 = np.concatenate([np.concatenate([train_images[a:], train_images[:a]]),
#                     np.concatenate([train_images[b:], train_images[:b]])])

# # y = to_categorical(y)


# y1 = (train_labels == np.concatenate([train_labels[a:], train_labels[:a]])).reshape(-1)
# y2 = (train_labels == np.concatenate([train_labels[b:], train_labels[:b]])).reshape(-1)
# y = np.concatenate([y1, y2])

y_tst = (test_labels == np.concatenate([test_labels[1:], test_labels[:1]])).reshape(-1)

In [20]:
# history = model.fit([X1, X2], y, epochs=500, batch_size=512, shuffle=True, class_weight={0:1, 1:2},
#          validation_data=([test_images, np.concatenate([test_images[1:], test_images[:1]])], y_tst))


In [21]:
# print_history_chart(history)

In [22]:
# model.save('siamese_vanilla_cifar10_diff.h5')

retrain

In [23]:
# !for i in {200..250}; do python retrain_cifar_10.py $i; done

evaluate

In [24]:
model = tf.keras.models.load_model("siamese_vanilla_cifar10_diff.h5")


In [25]:
y_pred = model.predict([test_images, np.concatenate([test_images[1:], test_images[:1]])])

In [26]:
accuracy_score(y_tst, y_pred>0.5)

0.93

In [27]:
confusion_matrix(y_tst, y_pred>0.5)

array([[8647,  361],
       [ 339,  653]])

In [28]:
f1_score(y_tst, y_pred>0.5)

0.6510468594217348

### models on the smaller datasets

#### 50% of the taining data

learning

In [ ]:
model = build_siamese_vanilla_model(train_images[0].shape, 10)

In [ ]:
train_images_50p = train_images[:len(train_images)//2]
train_labels_50p = train_labels[:len(train_labels)//2]

In [ ]:
# idx2 = make_pairs(train_labels_50p)
# idx3 = make_pairs(train_labels_50p)

# y = np.concatenate([train_labels_50p, train_labels_50p[idx2], train_labels_50p[idx3]])
# X1 = np.concatenate([train_images_50p, train_images_50p, train_images_50p])
# X2 = np.concatenate([train_images_50p, train_images_50p[idx2], train_images_50p[idx3]])
# y = to_categorical(y)

In [ ]:
# history = model.fit([X1, X2], y, epochs=300, batch_size=512, shuffle=True,
#          validation_data=([test_images, test_images], test_labels_cat))

In [ ]:
# model.save('siamese_vanilla_50p_cifar10.h5')

evaluate

In [ ]:
model = tf.keras.models.load_model("siamese_vanilla_50p_cifar10.h5")

In [ ]:
results_siamese_vanilla_50p = full_evaluate(model, [test_images, test_images], test_labels)

#### 10% of the taining data

learning

In [ ]:
# model = build_siamese_vanilla_model(train_images[0].shape, 10)

In [ ]:
# train_images_10p = train_images[:len(train_images)//10]
# train_labels_10p = train_labels[:len(train_labels)//10]

In [ ]:
# idx2 = make_pairs(train_labels_10p)
# idx3 = make_pairs(train_labels_10p)
# idx4 = make_pairs(train_labels_10p)
# idx5 = make_pairs(train_labels_10p)

# y = np.concatenate([train_labels_10p, train_labels_10p[idx2], train_labels_10p[idx3], train_labels_10p[idx4],
#                    train_labels_10p[idx5]])
# X1 = np.concatenate([train_images_10p, train_images_10p, train_images_10p, train_images_10p, train_images_10p])
# X2 = np.concatenate([train_images_10p, train_images_10p[idx2], train_images_10p[idx3], train_images_10p[idx4],
#                     train_images_10p[idx5]])
# y = to_categorical(y)

In [ ]:
# history = model.fit([X1, X2], y, epochs=300, batch_size=512, shuffle=True,
#          validation_data=([test_images, test_images], test_labels_cat))

In [ ]:
# model.save('siamese_vanilla_10p_cifar10.h5')

evaluate

In [ ]:
model = tf.keras.models.load_model("siamese_vanilla_10p_cifar10.h5")

In [ ]:
results_siamese_vanilla_10p = full_evaluate(model, [test_images, test_images], test_labels)

In [ ]:
# for k in range(1,10000):
#     print(k, end='\r')
#     model.predict([test_images, np.concatenate([test_images[:-k],test_images[:k]])])

In [ ]:
y_pred = np.argmax(model.predict([test_images, test_images]), 1)

acc = 0
c_m = None
flag = True

for i in range(1,11):
    true_sim = (test_labels == np.concatenate([test_labels[i:], test_labels[:i]])).reshape(-1)
    pred_sim = (y_pred == np.concatenate([y_pred[i:], y_pred[:i]]))
    acc += accuracy_score(true_sim, pred_sim)
    if flag:
        c_m = confusion_matrix(true_sim, pred_sim)
    else:
        c_m += confusion_matrix(true_sim, pred_sim)

acc = acc/10

In [ ]:
acc

In [ ]:
c_m